In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import pandas as pd
import uuid   # Unique identifier
import os
import time
import psycopg2
hostname = 'localhost'
database = 'employeeDB'
username = 'postgres'
pwd = 'admin'
port_id = 5432
conn = psycopg2.connect(host=hostname, dbname=database, user=username, password=pwd, port=port_id)
cur = conn.cursor()

C:\Users\Digvijay Singh Dhank\anaconda3\envs\drowsy\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !cd yolov5 && python train.py --img 320 --batch 16 --epochs 500 --data dataset.yml --weights yolov5s.pt --workers 2

In [5]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp/weights/last.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\Digvijay Singh Dhank/.cache\torch\hub\master.zip
YOLOv5  2022-5-28 Python-3.8.13 torch-1.8.2+cpu CPU

Fusing layers... 
Model summary: 213 layers, 7055974 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


In [6]:
cap = cv2.VideoCapture(0)
var = ""
employee_score = 95;
employee_name = "Bill\n"
employee_id = 123
while cap.isOpened():
    ret, frame = cap.read()
    cv2.rectangle(frame, (0, 0), (525, 73), (245, 117, 16), -1)
    cv2.putText(frame, "Employee Name:", (15, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, "Employee I.D.:",
                        (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, employee_name, (180, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, str(employee_id),
                        (180, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    # Make detections 
    results = model(frame)
    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    df = results.pandas().xyxy[0]
    d_count=df['name'].str.contains('drowsy').sum()
    if d_count>0:
        employee_score = employee_score - d_count
        score = employee_score.item()
        sql = "UPDATE employee SET score = array_append(score, %s) WHERE name = %s"
        var = (score,employee_name,)
        cur.execute(sql, var)
        
            
        
#         if d_count==1:
#             print("sleeping")
#         else:
#             print("awake")
    
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        conn.commit()
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
conn = psycopg2.connect(host=hostname, dbname=database, user=username, password=pwd, port=port_id)
cur = conn.cursor()

In [ ]:
cur.close()
conn.close()

In [ ]:
my_table    = pd.read_sql('select * from employee', conn)
print(my_table)